## Exploring the data

In [ ]:
import pandas as pd

In [ ]:
results_csv_path = "data/results.csv"
goalscorers_csv_path = "data/goalscorers.csv"
shootouts_csv_path = "data/shootouts.csv"

csv_paths = [results_csv_path, goalscorers_csv_path, shootouts_csv_path]

In [ ]:
results_df = pd.read_csv(results_csv_path, parse_dates=['date'])
goalscorers_df = pd.read_csv(goalscorers_csv_path, parse_dates=['date'])
shootouts_df = pd.read_csv(shootouts_csv_path, parse_dates=['date'])

In [ ]:
results_df.sample(5)

In [ ]:
goalscorers_df.columns

In [ ]:
shootouts_df.columns

In [ ]:
results_df.columns

In [ ]:
tournaments = results_df.tournament.unique().tolist()

# Tournaments by FIFA
FIFA_TOURNAMENTS = [t for t in tournaments if 'FIFA' in t]

print(FIFA_TOURNAMENTS)


In [ ]:
world_cup_matches = results_df[results_df['tournament'] == 'FIFA World Cup']

# Unique world cups
unique_world_cups = world_cup_matches['date'].dt.year.unique()
print(unique_world_cups)

In [ ]:
# Number of matches per world cup
world_cup_matches['date'].dt.year.value_counts().sort_index()

In [ ]:
goalscorers_df.columns

In [ ]:
# Merge goal scorers with world cup matches
goal_scorers_with_matches = world_cup_matches.merge(goalscorers_df, on=['date', 'home_team', 'away_team'])

goal_scorers_with_matches.shape

In [ ]:
goals_by_player = goal_scorers_with_matches[goal_scorers_with_matches['scorer'] == 'Miroslav Klose']

goals_by_player.date.unique().shape

In [ ]:
results_df.tournament.unique()

## Import the data to Neo4j

In [ ]:
#install neo4j library
!pip install neo4j

In [ ]:
import pandas as pd
import neo4j
from dotenv import load_dotenv
import os
from tqdm import tqdm
import logging

Since there are more than 48k matches in our database, we define a `BATCH_SIZE` parameter to ingest data in smaller samples. 

Then, we define a function called `create_indexes` that accepts a session object:

In [ ]:
# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
logger.info("Loading data...")

# Load data
results_df = pd.read_csv(results_csv_path, parse_dates=["date"])
goalscorers_df = pd.read_csv(goalscorers_csv_path, parse_dates=["date"])
shootouts_df = pd.read_csv(shootouts_csv_path, parse_dates=["date"])

# Set up Neo4j connection
uri = "neo4j://localhost"
user = ""
password = ""

try:
    driver = neo4j.GraphDatabase.driver(uri, auth=(user, password))
    print("Connected to Neo4j instance successfully!")
except Exception as e:
    print(f"Failed to connect to Neo4j: {e}")


BATCH_SIZE = 5000


Cypher indexes are database structures that improve query performance in Neo4j. They speed up the process of finding nodes or relationships based on specific properties. We need them for:

1. Faster query execution
2. Improved read performance on large datasets
3. Efficient pattern matching
4. Enforcing unique constraints
5. Better scalability as the database grows

In our case, indexes on team names, match IDs, and player names will help our queries run faster when searching for specific entities or performing joins across different node types. It is a best practice to create such indexes for your own databases.

Next, we have the `ingest_matches` function. It is large, so let's break it down block by block:

In [ ]:
def create_indexes(session):
    indexes = [
        "CREATE INDEX IF NOT EXISTS FOR (t:Team) ON (t.name)",
        "CREATE INDEX IF NOT EXISTS FOR (m:Match) ON (m.id)",
        "CREATE INDEX IF NOT EXISTS FOR (p:Player) ON (p.name)",
        "CREATE INDEX IF NOT EXISTS FOR (t:Tournament) ON (t.name)",
        "CREATE INDEX IF NOT EXISTS FOR (c:City) ON (c.name)",
        "CREATE INDEX IF NOT EXISTS FOR (c:Country) ON (c.name)",
    ]
    for index in indexes:
        session.run(index)
    print("Indexes created.")

`ingest_goals` and `ingest_shootouts` functions use similar constructs. However, `ingest_goals` have some additional error and missing value handling. 

At the end of the script, we have the `main()` function that executes all our ingestion functions with a session object:

In [ ]:
def ingest_matches(session, df):
    query = """
    UNWIND $batch AS row
    MERGE (m:Match {id: row.id})
    SET m.date = date(row.date), m.home_score = row.home_score, m.away_score = row.away_score, m.neutral = row.neutral
    MERGE (home:Team {name: row.home_team})
    MERGE (away:Team {name: row.away_team})
    MERGE (t:Tournament {name: row.tournament})
    MERGE (c:City {name: row.city})
    MERGE (country:Country {name: row.country})
    MERGE (home)-[:PLAYED_HOME]->(m)
    MERGE (away)-[:PLAYED_AWAY]->(m)
    MERGE (m)-[:PART_OF]->(t)
    MERGE (m)-[:PLAYED_IN]->(c)
    MERGE (c)-[:LOCATED_IN]->(country)
    WITH m, home, away, row.home_score AS hs, row.away_score AS as
    FOREACH(_ IN CASE WHEN hs > as THEN [1] ELSE [] END |
        MERGE (home)-[:WON]->(m)
        MERGE (away)-[:LOST]->(m)
    )
    FOREACH(_ IN CASE WHEN hs < as THEN [1] ELSE [] END |
        MERGE (away)-[:WON]->(m)
        MERGE (home)-[:LOST]->(m)
    )
    FOREACH(_ IN CASE WHEN hs = as THEN [1] ELSE [] END |
        MERGE (home)-[:DREW]->(m)
        MERGE (away)-[:DREW]->(m)
    )
    """
    for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Ingesting matches"):
        batch = df.iloc[i : i + BATCH_SIZE]
        data = []
        for _, row in batch.iterrows():
            match_data = {
                "id": f"{row['date']}_{row['home_team']}_{row['away_team']}",
                "date": row["date"].strftime("%Y-%m-%d"),
                "home_score": int(row["home_score"]),
                "away_score": int(row["away_score"]),
                "neutral": bool(row["neutral"]),
                "home_team": row["home_team"],
                "away_team": row["away_team"],
                "tournament": row["tournament"],
                "city": row["city"],
                "country": row["country"],
            }
            data.append(match_data)
        session.run(query, batch=data)


def ingest_goals(session, df):
    query = """
    UNWIND $batch AS row
    MATCH (m:Match {id: row.id})
    MERGE (p:Player {name: row.scorer})
    MERGE (t:Team {name: row.team})
    CREATE (p)-[s:SCORED_FOR]->(t)
    SET s.own_goal = row.own_goal,
        s.penalty = row.penalty
    FOREACH(_ IN CASE WHEN row.minute IS NOT NULL THEN [1] ELSE [] END |
        SET s.minute = row.minute
    )
    CREATE (p)-[r:SCORED_IN]->(m)
    SET r.own_goal = row.own_goal,
        r.penalty = row.penalty
    FOREACH(_ IN CASE WHEN row.minute IS NOT NULL THEN [1] ELSE [] END |
        SET r.minute = row.minute
    )
    """
    for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Ingesting goals"):
        batch = df.iloc[i : i + BATCH_SIZE]
        data = []
        for _, row in batch.iterrows():
            try:
                goal_data = {
                    "id": f"{row['date']}_{row['home_team']}_{row['away_team']}",
                    "scorer": (
                        row["scorer"]
                        if not pd.isna(row["scorer"])
                        else "Unnamed Player"
                    ),
                    "team": row["team"],
                    "minute": (
                        float(row["minute"]) if pd.notnull(row["minute"]) else None
                    ),
                    "own_goal": bool(row["own_goal"]),
                    "penalty": bool(row["penalty"]),
                }
                data.append(goal_data)
            except Exception as e:
                logger.error(f"Error processing row: {row}")
                logger.error(f"Error details: {str(e)}")

        if data:
            try:
                session.run(query, batch=data)
            except Exception as e:
                logger.error(f"Error executing batch: {str(e)}")
                logger.error(f"Problematic batch: {data}")


def ingest_shootouts(session, df):
    query = """
    UNWIND $batch AS row
    MATCH (m:Match {id: row.id})
    MATCH (w:Team {name: row.winner})
    MERGE (m)-[s:HAD_SHOOTOUT]->(w)
    SET s.winner = row.winner
    FOREACH(_ IN CASE WHEN row.first_shooter IS NOT NULL THEN [1] ELSE [] END |
        SET s.first_shooter = row.first_shooter
    )
    """
    for i in tqdm(range(0, len(df), BATCH_SIZE), desc="Ingesting shootouts"):
        batch = df.iloc[i : i + BATCH_SIZE]
        data = []
        for _, row in batch.iterrows():
            shootout_data = {
                "id": f"{row['date']}_{row['home_team']}_{row['away_team']}",
                "winner": row["winner"],
                "first_shooter": (
                    row["first_shooter"] if pd.notnull(row["first_shooter"]) else None
                ),
            }
            data.append(shootout_data)
        session.run(query, batch=data)

### Run to import all data

In [ ]:
def main():
    with driver.session() as session:
        create_indexes(session)
        ingest_matches(session, results_df)
        ingest_goals(session, goalscorers_df)
        ingest_shootouts(session, shootouts_df)

    print("Data ingestion completed!")
    driver.close()


if __name__ == "__main__":
    main()